### Create PNG slices from TIFF for each MECE mask

In [1]:
import os, sys, subprocess
import psycopg2
import numpy as np
from PIL import Image, ImageFilter
from subprocess import call
from natsort import natsorted
from skimage import io, transform
import warnings; warnings.filterwarnings("ignore")

#### Collect MECE tiff stacks

In [15]:
tiffs_dir_path = "./RegionTiffs/"
tiff_stacks = natsorted(os.listdir(tiffs_dir_path))
tiff_stacks = [tiff.replace('.tif', '') for tiff in tiff_stacks]
tiff_stacks

['Forebrain - Diencephalon - Anterior group of the posterior tubercular vmat2 neurons',
 'Forebrain - Diencephalon - Anterior preoptic dopaminergic cluster',
 'Forebrain - Diencephalon - Anterior pretectum cluster of vmat2 Neurons',
 'Forebrain - Diencephalon - Dopaminergic Cluster 1 - ventral thalamic and periventricular posterior tubercular DA neurons',
 'Forebrain - Diencephalon - Dopaminergic Cluster 2 - posterior tuberculum',
 'Forebrain - Diencephalon - Dopaminergic Cluster 3 - hypothalamus',
 'Forebrain - Diencephalon - Dopaminergic Cluster 4-5 - posterior tuberculum and hypothalamus',
 'Forebrain - Diencephalon - Dopaminergic Cluster 6 - hypothalamus',
 'Forebrain - Diencephalon - Dopaminergic Cluster 7 - Caudal Hypothalamus',
 'Forebrain - Diencephalon - Eminentia Thalami',
 'Forebrain - Diencephalon - Hypothalamus 6.7FRhcrtR-Gal4 cluster 1',
 'Forebrain - Diencephalon - Hypothalamus 6.7FRhcrtR-Gal4 cluster 2',
 'Forebrain - Diencephalon - Hypothalamus - Caudal Hypothalamus Ne

In [17]:
mece_dir_path = "../app/assets/images/8-MeceMasks/"

#### Output them as PNG slices

In [18]:
def slice_tiff_stacks(tiff_stacks, input_tiffs_dir, output_pngs_dir):
    for tiff in tiff_stacks:
        input_tiff = input_tiffs_dir + tiff + '.tif'
        output_pngs = output_pngs_dir + tiff
        
        try:
            os.mkdir(output_pngs)
        except:
            print('directory exists')
        finally:
            stack = io.imread(input_tiff)
            stack = [np.rot90(im) for im in stack]
            
        for i, image in enumerate(stack):
            io.imsave("{0}/{1}-{2}.png".format(output_pngs, tiff, i), image)

In [19]:
slice_tiff_stacks(tiff_stacks, tiffs_dir_path, mece_dir_path)

#### Use PIL to trim pngs to outline pngs

In [23]:
mece_dir_path = "../app/assets/images/8-MeceMasks/"

In [24]:
def convert_masks_to_outlines(input_pngs_dir):
    stack_dirs = [d for d in os.listdir(input_pngs_dir) if '.DS_Store' not in d]
    for stack in stack_dirs:
        stack_dir_path = os.path.join(input_pngs_dir, stack)
        slices = [s for s in os.listdir(stack_dir_path) if '.DS_Store' not in s]
        for sli in slices:
            sli_path = os.path.join(stack_dir_path, sli)
            sli_image = Image.open(sli_path).convert("L")
            sli_image = sli_image.filter(ImageFilter.FIND_EDGES)
            sli_image.save(sli_path)
            

In [25]:
convert_masks_to_outlines(mece_dir_path)

KeyboardInterrupt: 

#### Use ImageMagick to convert png to transparent

In [38]:
def convert_masks_to_transparent(input_pngs_dir):
    stack_dirs = [d for d in os.listdir(input_pngs_dir) if '.DS_Store' not in d]
    for stack in stack_dirs:
        stack_dir_path = os.path.join(input_pngs_dir, stack)
        print(stack_dir_path)
        try:
            subprocess.check_output(["mogrify", "-transparent", "black", "{0}/*.png".format(stack_dir_path)], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)

In [39]:
convert_masks_to_transparent(mece_dir_path)

../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 18
../app/assets/images/8-MeceMasks/Hindbrain - Isl1 Stripe 1
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 20
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 16
../app/assets/images/8-MeceMasks/Hindbrain - Glyt2 Cluster 12
../app/assets/images/8-MeceMasks/Hindbrain - Isl1 Cluster 2
../app/assets/images/8-MeceMasks/Hindbrain - Cerebellum Gad1b Enriched Areas
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 11
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Stripe 2
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 8
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Oxtl Cluster 4 - sparse in hypothalamus
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Migrated Area of the Pretectum (M1)
../app/assets/images/8-MeceMasks/Hindbrain - Gad1b Cluster 6
../app/assets/images/8-MeceMasks/Midbrain - Ptf1a Cluster
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon 

### Prepare postgreSQL table

In [111]:
create_table_command = (
"""
    DROP TABLE IF EXISTS meceMasks;
    CREATE TABLE meceMasks (
        mask_img_id SERIAL PRIMARY KEY,
        mask_name TEXT NOT NULL,
        mask_id INT NOT NULL,
        mask_image_path TEXT NOT NULL,
        mask_image_size INT
    )
""")

#### Connect to zbrain2db database and create table

In [160]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

#### Prepare table entries for all mece masks

In [161]:
mece_dir_path = "../app/assets/images/8-MeceMasks/"
mece_dirs = [d for d in os.listdir(mece_dir_path) if '.DS_Store' not in d]
mece_dirs = [os.path.join(mece_dir_path, d) for d in mece_dirs] 

In [162]:
slice_id = 2000
insert_commands = list()
for i, mece_dir in enumerate(mece_dirs):
    slices = natsorted([d for d in os.listdir(mece_dir) if '.DS_Store' not in d])
    mask_name = mece_dir.split('/')[-1]
    for j, slice_path in enumerate(slices):
        slice_size = os.stat(os.path.join(mece_dir,slice_path)).st_size
    
        insert_command = ("INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (%i, '%s', %i, '%s', %i)"
                    % (slice_id, mask_name.replace("'", "''"), i, slice_path.replace("'", "''"), slice_size))
        slice_id += 1
    
        insert_commands.append(insert_command)

In [163]:
print(insert_commands[:5])

["INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (2000, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-0.png', 927)", "INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (2001, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-1.png', 927)", "INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (2002, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-2.png', 927)", "INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (2003, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-3.png', 927)", "INSERT INTO meceMasks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (2004, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-4.png', 927)"]


In [167]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

duplicate key value violates unique constraint "mecemasks_pkey"
DETAIL:  Key (mask_img_id)=(2000) already exists.

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction block

current transaction is aborted, commands ignored until end of transaction b

#### Query database

In [165]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM meceMasks ORDER BY mask_img_id')

rows = cursor.fetchall()
for row in rows[0:138]:
    print(row)
cursor.close()

(2000, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-0.png', 927)
(2001, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-1.png', 927)
(2002, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-2.png', 927)
(2003, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-3.png', 927)
(2004, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-4.png', 927)
(2005, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-5.png', 927)
(2006, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-6.png', 927)
(2007, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-7.png', 927)
(2008, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-8.png', 927)
(2009, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-9.png', 927)
(2010, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Cluster 18-10.png', 927)
(2011, 'Hindbrain - Gad1b Cluster 18', 0, 'Hindbrain - Gad1b Clu